In [1]:
# imports
from imgaug import augmenters as iaa
import imgaug as ia
import os
import numpy as np
import cv2
import glob
from PIL import Image
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


In [2]:
# seed for ia randomizer
ia.seed(1)


In [3]:
# fetch certain file types
images_names = []
ignored = 0

for fname in os.listdir('images'):
    if fname.endswith('.jpg'):
        images_names.append(fname)
    else:
        ignored += 1

print("found", len(images_names), ".jpg files")
print("ignored", ignored, "other file types")


found 9 .jpg files
ignored 9 other file types


In [4]:
# extract only bounding box information from .txt file (see format below)
def extract_bounding_boxes(file):
    bounding_boxes = []
    with open(file) as f:
        boxes = f.readlines()
        for box in boxes:
            box_arr = box.split(' ')
            bounding_boxes.append(
                [box_arr[1], box_arr[2], box_arr[3], box_arr[4].strip("\n")])
    return bounding_boxes

# extract only the classification label from .txt file (see format below)


def extract_classification_labels(file):
    class_labels = []
    with open(file) as f:
        labels = f.readlines()
        for label in labels:
            lab_arr = label.split(' ')
            class_labels.append(lab_arr[0])
    return class_labels


Bounding box .txt files should look as follows:

3 0.297656 0.615278 0.118750 0.205556   
3 0.622656 0.374306 0.056250 0.093056

Where the first number represents a classification label

In [5]:
# put images in np-array from memory & put corresponding bounding boxes in array in order
# bounding_box_images contains tuples in which the first element corresponds to the image and the second element corresponds to the bounding boxes details as an array
bounding_box_images = []
classification_labels = []

for image_name in images_names:
    # obtain image
    location = "images/"+image_name
    image = cv2.imread(location)

    # obtain bounding boxes
    loc = os.path.splitext(location)[0] + '.txt'
    bounding_boxes = extract_bounding_boxes(loc)

    # put data in tuple and append to array
    bounding_box_images.append((image, bounding_boxes))

    # save classification labels for future use
    classification_labels.append(extract_classification_labels(loc))


print("images found with bounding box(es):", len(bounding_box_images))
print("classifications found for boxes:", len(classification_labels))
print(classification_labels)


images found with bounding box(es): 9
classifications found for boxes: 9
[['3', '3'], ['3', '0'], ['0'], ['3'], ['0'], ['3', '3'], ['3', '3', '3', '3', '3'], ['0'], ['3']]


In [6]:
# sequential operations

seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)),  # change brightness, doesn't affect BBs
    iaa.Affine(
        translate_px={"x": 40, "y": 60},
        scale=(0.5, 0.7)
    )  # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
])


In [7]:
# Augment BBs and images.

# function takes image with corresponding bounding boxes and performs augmentation on both.
def augment_bb_image(img, bbs):
    x, y = seq(image=img, bounding_boxes=bbs)
    return x, y

# transforms pixel data from bounding boxes to normalized yolo values


def normalize_bounding_boxes(img_shape, bbs):
    width = img_shape[0]
    heigth = img_shape[1]
    normalized_bbs = []
    for bb in bbs:
        bb_normalized = [round(bb[0]/width, 6), round(bb[1]/heigth, 6),
                         round(bb[2]/width, 6), round(bb[3]/heigth, 6)]
        normalized_bbs.append(bb_normalized)
    return normalized_bbs


In [22]:
# perform image & bounding box augmentation on all images
augmented_bb_images = []
for bb_img in bounding_box_images:
    # print(len(bb_img[1]))
    # extract bounding boxes per image and create BoundingBox objects from data
    bbs = BoundingBoxesOnImage([
        BoundingBox(x1=bb[0], y1=bb[1], x2=bb[2], y2=bb[3]) for bb in bb_img[1]
    ], shape=bb_img[0].shape)

    # augmented_bb_images
    aug_img, aug_bbs = augment_bb_image(bb_img[0], bbs)

    # normalize bounding boxes to yolo format
    aug_bbs = [[bb.x1, bb.y1, bb.x2, bb.y2] for bb in aug_bbs]
    aug_bbs = normalize_bounding_boxes(bb_img[0].shape, aug_bbs)
    aug_bb_img = [aug_img, aug_bbs]
    augmented_bb_images.append(aug_bb_img)


# add classification label to each box
for idx, images in enumerate(augmented_bb_images):
    for idx2, box in enumerate(images[1]):
        label = classification_labels[idx][idx2]
        augmented_bb_images[idx][1] = np.concatenate((label, box), axis=None)
        print(augmented_bb_images[idx][1])



['3' '0.339543' '0.136804' '0.339712' '0.137022']
['3' '0.339484' '0.136744' '0.34002' '0.136894']
['3' '0.489656' '0.184267' '0.490197' '0.184474']
['0' '0.489763' '0.184248' '0.490061' '0.184286']
['0' '0.392805' '0.153757' '0.393022' '0.153998']
['3' '0.447974' '0.171057' '0.448069' '0.171363']
['0' '0.469294' '0.177916' '0.46943' '0.178075']
['3' '0.328478' '0.133256' '0.328853' '0.133471']
['3' '0.32863' '0.13331' '0.329156' '0.133692']
['3' '0.448872' '0.171328' '0.448982' '0.171382']
['3' '0.448893' '0.171465' '0.448929' '0.171481']
['3' '0.448831' '0.171371' '0.449048' '0.171558']
['3' '0.449001' '0.171409' '0.449281' '0.17164']
['3' '0.448787' '0.171424' '0.448858' '0.171584']
['0' '0.322539' '0.131425' '0.322644' '0.131532']
['3' '0.363694' '0.144378' '0.363734' '0.144425']


In [24]:
# augmented_bb_images folder path
path = 'augmented_bb_images/'


In [25]:
# ONLY EXECUTE THIS CELL BLOCK IF YOU WANT THE CURRENT AUGMENTED IMAGES FOLDER TO BE DELETED!!!

# empty augmented_images folder

for f in os.listdir(path):
    os.remove(os.path.join(path, f))


In [28]:
# save augmented images & corresponding bounding boxes in the 'augmented_bb_images' folder

for index, img in enumerate(augmented_bb_images):

    try:
        # save augmented image
        name = path+images_names[index]
        cv2.imwrite(os.path.join(path, name), img[0])

        # save augmented bounding box
        name = path + os.path.splitext(images_names[index])[0] + '.txt'
        print(img[1])
        np.savetxt(name, img[1])
        print("succesfully saved image with index:", str(index))
    except Exception as ex:
        print(ex)


# TODO: FIX DAT .TXT FILES WORDEN OPGESLAGEN MET JUISTE BOXES (MISMATCH ERROR) OOK MEERDERE BOXES PER IMAGE!!

['3' '0.339484' '0.136744' '0.34002' '0.136894']
Mismatch between array dtype ('<U32') and format specifier ('%.18e')
['0' '0.489763' '0.184248' '0.490061' '0.184286']
Mismatch between array dtype ('<U32') and format specifier ('%.18e')
['0' '0.392805' '0.153757' '0.393022' '0.153998']
Mismatch between array dtype ('<U32') and format specifier ('%.18e')
['3' '0.447974' '0.171057' '0.448069' '0.171363']
Mismatch between array dtype ('<U32') and format specifier ('%.18e')
['0' '0.469294' '0.177916' '0.46943' '0.178075']
Mismatch between array dtype ('<U32') and format specifier ('%.18e')
['3' '0.32863' '0.13331' '0.329156' '0.133692']
Mismatch between array dtype ('<U32') and format specifier ('%.18e')
['3' '0.448787' '0.171424' '0.448858' '0.171584']
Mismatch between array dtype ('<U32') and format specifier ('%.18e')
['0' '0.322539' '0.131425' '0.322644' '0.131532']
Mismatch between array dtype ('<U32') and format specifier ('%.18e')
['3' '0.363694' '0.144378' '0.363734' '0.144425']
Mi